In [1]:
# If the library is not installed - unlock the field (for download from google disk)
! pip install gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gdown: filename=gdown-4.5.3-py3-none-any.whl size=14821 sha256=4a995be4ae276e2db883fe5c7954f6f93dd78c42304dc2676f0da1af7de5391d
  Stored in directory: /root/.cache/pip/wheels/94/8d/0b/bdcd83555c3555f91a33f6c2384428d9f163c7d75ab0d272b4
Successfully built gdown


In [2]:
# Download lides_data.zip
!gdown --id 1CqOrYqsj_wb4NKNBNF7Cc_cZ9WuF7T9Q

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1CqOrYqsj_wb4NKNBNF7Cc_cZ9WuF7T9Q
To: /kaggle/working/likes_data.zip
100%|████████████████████████████████████████| 362M/362M [00:04<00:00, 81.8MB/s]


In [3]:
# Unzip lides_data.zip
!unzip likes_data.zip

Archive:  likes_data.zip
  inflating: test                    
  inflating: __MACOSX/._test         
  inflating: track_artists.csv       
  inflating: train                   
  inflating: __MACOSX/._train        


In [4]:
# Import lib 
import numpy as np
import gc
import os
import pandas as pd
# import tqdm

from tqdm.notebook import tqdm
from sklearn.model_selection import KFold

In [5]:
# Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, 
#где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

In [6]:
# Create df from data
columns_user_id = []
columns_items_id = []

with open('train') as f:
    
    lines = f.readlines()
    
    for user, line in enumerate(tqdm(range(len(lines)))):
        items_likes = [int(i) for i in lines[line].strip().split(' ')][::-1]
        user_id = [user] * len(items_likes)
        columns_user_id.extend(user_id)
        columns_items_id.extend(items_likes)
            
df = pd.DataFrame({
    'user_id': columns_user_id,
    'items_id': columns_items_id
})

  0%|          | 0/1160084 [00:00<?, ?it/s]

In [7]:
df.head()

,user_id,items_id
0,0,388242
1,0,278503
2,0,102795
3,0,470957
4,0,159637


In [8]:
df['order'] = df.groupby('user_id').cumcount() 
df.head()

,user_id,items_id,order
0,0,388242,0
1,0,278503,1
2,0,102795,2
3,0,470957,3
4,0,159637,4


In [9]:
df.to_csv('data.csv')

In [10]:
# Итоговая задача - построить схему валидации для данного соревнования с учетом особенностей сорвенования

#     Между train и test не должно быть общих пользователей
#     Количество фолдов задается через параметр класса n_folds
#     В test должно быть не более p последних треков (параметр класса p)

In [11]:
class UsersKFoldPOut():
    
    def __init__(self, n_folds: int, p: int, random_seed: int=23):
        self.n_folds = n_folds
        self.p = p
        self.rd_seed = random_seed
        
    def split(self, df: pd.DataFrame):
        
        list_users = df['user_id'].unique()
        kf = KFold(n_splits=self.n_folds, 
                   shuffle=True,
                   random_state=self.rd_seed)
        
        for train_index, test_index in kf.split(list_users):
            # Kfold возвращает индексы юзеров. Создадим списки с users_id
            train_users, test_users = np.take(list_users, train_index), np.take(list_users, test_index)
            # Получим наше разбитие на тренировочный и валидационный
            mask_train = df['user_id'].isin(train_users)
            mask_test = (df['user_id'].isin(test_users)) & (df['order'] < self.p)
            yield mask_train, mask_test
            


In [14]:
p = 1
cv = UsersKFoldPOut(n_folds=3, p=p)
interactions = df

for i, (train_mask, test_mask) in enumerate(cv.split(interactions)):
    
    train = interactions[train_mask]
    test = interactions[test_mask]
    
    total_users = set(train['user_id'].unique()).intersection(test['user_id'].unique())
    
    # Проверка на отсуствие совпадений юзеров в train и val частях
    check_1 = (
        len(total_users) == 0,
        f"Users {total_users} in train and test"
    )
    # Проверка на условие что кол-во значений не превышает p 
    check_2 = (
        test['user_id'].values.max() <= p,
        'More than p tracks for user in test'
    )
    assert check_1
    assert check_2
    
    
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 62769950, Test: 386695
Fold#1 | Train: 62749697, Test: 386695
Fold#2 | Train: 62857621, Test: 386694
